# Project Part 1

#### Importing Modules

In [ ]:
import os
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split
import itertools

from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Flatten, Dropout, concatenate, Input, Conv2D, MaxPooling2D
from keras.optimizers import Adam, Adadelta
from keras.layers.advanced_activations import LeakyReLU
from keras.utils.np_utils import to_categorical

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
#!pip install opencv-python

#### Viewing data

In [ ]:
import os
rootdir='F:\\GreatLearning\\AI\\ComputerVision\\Project\\plant_seeding_classification_data\\train\\'

for file in os.listdir(rootdir):
    d = os.path.join(rootdir, file)
    if os.path.isdir(d):
        print(file)



- We have 12 species of plants avaialable for training

In [ ]:
for file in os.listdir(rootdir):
    d = os.path.join(rootdir, file)
    if os.path.isdir(d):
        print(file)



In [ ]:
import glob
os.chdir(d)
count=0
imgList=[]
for file in glob.glob("*.PNG"):
    print(file)
    count+=1
    imgList.append(file)
    if (count==10):
        break

- From first glance, images are in png format

In [ ]:
imgList

In [ ]:
img1= os.path.join(d, imgList[0])

import imageio
import matplotlib.pyplot as plt
%matplotlib inline

#function created for viewing basic data features of images
def imgBasics(path,imgName):
    img1= os.path.join(path, imgName)
    pic = imageio.imread(img1)
    plt.figure(figsize = (5,5))
    plt.imshow(pic)

    #Basic properties of image
    print('Type of the image : ' , type(pic)) 
    print('Shape of the image : {}'.format(pic.shape)) 
    print('Image Hight {}'.format(pic.shape[0])) 
    print('Image Width {}'.format(pic.shape[1])) 
    print('Dimension of Image {}'.format(pic.ndim))
    print('Image size {}'.format(pic.size)) 
    print('Maximum RGB value in this image {}'.format(pic.max())) 
    print('Minimum RGB value in this image {}'.format(pic.min()))
    print('Value of only R channel {}'.format(pic[ 100, 50, 0])) 
    print('Value of only G channel {}'.format(pic[ 100, 50, 1])) 
    print('Value of only B channel {}'.format(pic[ 100, 50, 2]))

In [ ]:
imgBasics(d,imgList[0])


In [ ]:
imgBasics(d,imgList[1])

In [ ]:
imgBasics(d,imgList[2])

In [ ]:
imgBasics(d,imgList[9])

In [ ]:
pic = imageio.imread(img1) 
gray = lambda rgb : np.dot(rgb[... , :3] , [0.299 , 0.587, 0.114]) 
gray = gray(pic) 
plt.figure( figsize = (5,5))  
plt.imshow(gray, cmap = plt.get_cmap(name = 'gray')) 
plt.show()

In [ ]:
import numpy as np 
pic = imageio.imread(img1) 
fig, ax = plt.subplots(nrows = 1, ncols=3, figsize=(15,5))  
for c, ax in zip(range(3), ax):     
     split_img = np.zeros(pic.shape, dtype="uint8")       
     split_img[ :, :, c] = pic[ :, :, c]     
     ax.imshow(split_img)

- Images are of very diferent resolutions, ranging from 1023x1023 to 481x481
- they have 3 channnels (RGB)
- Lot of details avialable apart from just the plant species like sand grains or stone or scales

### Loading Data

#### Exploring the folders containing data

In [ ]:
#Understaanding data for loading
train_dir = 'F:/GreatLearning/AI/ComputerVisionProject/plant_seeding_classification_data/train'
test_dir  = 'F:/GreatLearning/AI/ComputerVisionProject/plant_seeding_classification_data/test'
sample_submission = pd.read_csv('F:\GreatLearning\AI\ComputerVision\Project\plant_seeding_classification_data\sample_submission.csv')

In [ ]:
def get_immediate_subdirectories(a_dir):
    return [name for name in os.listdir(a_dir)
            if os.path.isdir(os.path.join(a_dir, name))]

In [ ]:
specPath='F:\\GreatLearning\\AI\\ComputerVision\\Project\\plant_seeding_classification_data\\train\\'
specPathTest='F:\\GreatLearning\\AI\\ComputerVision\\Project\\plant_seeding_classification_data\\test\\'
cat_Folder_list=get_immediate_subdirectories(specPath)

#### List of species in the train folders

In [ ]:
Plant_Species=cat_Folder_list
print('List od Plant species: ', Plant_Species)

#### Nummber of images in the train sub folders along with categories

In [ ]:
#No. of images under each plant species foler for train
for img in Plant_Species:
    print('{}   -->   {} training images'.format(img, len(os.listdir(os.path.join(specPath, img)))))

- We have 12 species and all flowers have varying data for training
- Loose silky-bent has highest concentration of images having 671 images and common wheat has of 221 images

### Loading data from all folders along with mapped categories

- Importing data
- Preprocessing the image for using it in models
- For Supervised Models like KNN, 
    - importing and resizing it to 32x32 with RGB to maintain its colours and pattern (COLOR_BGR2RGB)
    - For Interpolation using cv.INTER_AREA
    - dividing the values with 255 to normalize it and make it float
    - capturing the folder names as categories
    

In [ ]:
#we can not directly use the image, we have to process the image.

from pathlib import Path
from skimage.io import imread
from keras.preprocessing import image
import cv2 as cv
def load_image_files(container_path):
    image_dir = Path(container_path)
    folders = [directory for directory in image_dir.iterdir() if directory.is_dir()]
    categories = [fo.name for fo in folders]

    images = []
    flat_data = []
    target = []
    count = 0
    train_img = []
    label_img = []
    for i, direc in enumerate(folders):
        for file in direc.iterdir():
            count += 1
            img = imread(file)
            img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
            img_pred = cv.resize(img, (32, 32), interpolation=cv.INTER_AREA)
            img_pred = image.img_to_array(img_pred)
            img_pred = img_pred / 255
            train_img.append(img_pred)
            label_img.append(categories[i])
            
    X = np.array(train_img)
    y = np.array(label_img)
    return X,y

#Using the Keras pre-processing library the image is converted to an array and then normalised.

In [ ]:
X = []
y = []
X,y = load_image_files(specPath)

##### Exploring shape of imported data

In [ ]:
X.shape

In [ ]:
y.shape

#### Exploring images captured

In [ ]:
y[0]

In [ ]:
plt.imshow(X[0])
plt.show()

In [ ]:
plt.imshow(X[0],cmap='gist_earth')
plt.show()

In [ ]:
fig=plt.figure(figsize=(15,15))

for i in range(1,101):
  img=X[i]
  fig.add_subplot(10,10,i)
  plt.imshow(img,cmap='gray')

plt.show()
print('Label: ', y[1:101])

In [ ]:
fig=plt.figure(figsize=(15,15))

for i in range(1,101):
  img=X[1000+i]
  fig.add_subplot(10,10,i)
  plt.imshow(img)

plt.show()
print('Label: ', y[1001:1101])

#### Image analysis 
- Images are not simple 
- Contains foreground and background details with multiple objects as noise in it like rulers, stones, soil, sand, corners, alphabets
- Huge variations of data present
- Images are not focused as well
- Actual plants are covering very less pixels compared to background and noise. This will lead to high class imbalance for target object and noise.
- Simple supervised models will have hard time filtering the actual plants with soil and stones as they see the whole picture as a single input and are not splitting foreground from background.

### Creating data sets for training and testing

- Splitting Whole data set to Train, Val and Test with 80%, 10%, 10% respectively

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2)

In [ ]:
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, random_state=42, test_size=0.5)

In [ ]:
#View data set shape
print("X_train: "+str(X_train.shape))
print("X_test: "+str(X_test.shape))
print("X_val: "+str(X_val.shape))
print("y_train: "+str(y_train.shape))
print("y_test: "+str(y_test.shape))
print("y_val: "+str(y_val.shape))

In [ ]:
#View Raw data in train set
X_train[0]

In [ ]:
#Reshaping Data sets for using in KNN model

from builtins import range
from builtins import object

num_training = X_train.shape[0]
mask = list(range(num_training))
X_train = X_train[mask]
y_train = y_train[mask]

num_test = X_test.shape[0]
mask = list(range(num_test))
X_test = X_test[mask]
y_test = y_test[mask]

num_val = X_val.shape[0]
mask = list(range(num_val))
X_val = X_val[mask]
y_val = y_val[mask]

# Reshape the image data into rows
X_train = np.reshape(X_train, (X_train.shape[0], -1))
X_test = np.reshape(X_test, (X_test.shape[0], -1))
X_val = np.reshape(X_val, (X_val.shape[0], -1))

print("X_train: "+str(X_train.shape))
print("X_test: "+str(X_test.shape))
print("X_val: "+str(X_val.shape))
print("y_train: "+str(y_train.shape))
print("y_test: "+str(y_test.shape))
print("y_val: "+str(y_val.shape))

In [ ]:
print(y.view())

#### Image classification with KNN

##### KNN
- For Plant species raw data, expectations is the leaf's and other patern remain close to each other and a KNN model will be able pick up the common features and group it together
- k-nearest neighbor algorithm is for classifying objects based on closest training examples in the feature space. k-nearest neighbor algorithm is among the simplest of all machine learning algorithms. Training process for this algorithm only consists of storing feature vectors and labels of the training images. In the classification process, the unlabelled query point is simply assigned to the label of its k nearest neighbors.
- A main advantage of the KNN algorithm is that it performs well with multi-modal classes because the basis of its decision is based on a small neighborhood of similar objects. Therefore, even if the target class is multi-modal, the algorithm can still lead to good accuracy. 

In [ ]:
from sklearn.metrics import accuracy_score,confusion_matrix,recall_score,f1_score,precision_score,roc_curve,log_loss,auc
from sklearn.neighbors import KNeighborsClassifier


#KNN Model with 1 neighbour

KnnModel = KNeighborsClassifier(n_neighbors=1)
KnnModel.fit(X_train,y_train)
y_predict=KnnModel.predict(X_test)



In [ ]:
print('Accuracy score:',accuracy_score(y_test,y_predict))
print('confuion matrix:\n',confusion_matrix(y_test,y_predict))


- With single neighbour we are able to acheive close to 30% accuracy, but 1 neighbour is higly volatile and wont give us generalised result

In [ ]:
# Initializing the value of k and finding the accuracies on validation data
k_vals = range(1, 30, 2)
accuracies = []

for k in range(1, 30, 2):
  knn = KNeighborsClassifier(n_neighbors=k)
  knn.fit(X_train, y_train)
  score = knn.score(X_val, y_val)
  print("k value=%d, accuracy score=%.2f%%" % (k, score * 100))
  accuracies.append(score)
 
# finding the value of k which has the largest accuracy
i = int(np.argmax(accuracies))
print("k=%d value has highest accuracy of %.2f%% on validation data" % (k_vals[i],accuracies[i] * 100))

- Even though we got highest accuracies at 1 neighbour but we will go ahead with k=25 for more generalized approach which showed similar high accuracies on validation data set.

In [ ]:
knn = KNeighborsClassifier(n_neighbors=25)
knn.fit(X_train, y_train)
predictions = knn.predict(X_test)


In [ ]:
print("EVALUATION ON TESTING DATA")
print(confusion_matrix(y_test,predictions))
print(knn.score(X_test, y_test))

In [ ]:
plt.figure(figsize=(2,2))
plt.imshow(X_test[59].reshape(32,32,3))
plt.show()
image = X_test[59]
print('Prediction:',knn.predict(image.reshape(1, -1)))
print('Actual:',y_test[59])

In [ ]:
plt.figure(figsize=(2,2))
plt.imshow(X_test[300].reshape(32,32,3))
plt.show()
image = X_test[300]
print('Prediction:',knn.predict(image.reshape(1, -1)))
print('Actual:',y_test[300])

In [ ]:

predictions = knn.predict(X_test)
print(classification_report(y_test, predictions))

In [ ]:
print(knn.score(X_test, y_test))

- Accuracies from KNN are close to 24%.
- We can observe on each classes precision and recall are very low.
- Model is not able to identify and split relevent data from rest of the noise.

- For Plant species raw data, expectations is the leaf's and other patern remain close to each other and a KNN model will be able pick up the common features and group it together.
- While data analysis we noticed the images contains lot of noise or i few images actual plat is hardly cvering 5% of total pixels
- Executed KNN from 1 to 30 neighbours and identified best values were at 1 neighbour and 25 neighbours. We chose for 25 neighbours so that we will get a more generalized prediction on classification.
- With KNN our classification accuracies were close to 20% which are way below acceptable levels
- We can observe there is an underlying pattern to the images for both raw pixel intensities and color. KNN is not capable enough to understand the differences and classifymore accurately.

##### Issues with KNN
- KNN depends on nearest neighbours, which might not be the best choice all the time. Observed the same issue during our evaluation proces as well
- a major disadvantage of the KNN algorithm is that it uses all the features equally in computing for similarities. This can lead to classification errors, especially when there is only a small subset of features that are useful for classification. 
- While data analysis we noticed the images contains lot of noise or in few images actual plant is hardly cvering 5% of total pixels

#### Image classification with Neural Network

- Using One hot encoder to convert the categories to array formats

In [ ]:
#from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import OneHotEncoder


one_hot_encoder = OneHotEncoder(sparse=False)
one_hot_encoder.fit(y_train.reshape(-1, 1))

y_train = one_hot_encoder.transform(y_train.reshape(-1, 1))
#y_train = pd.DataFrame(data=y_train, columns=one_hot_encoder.categories_)

y_test = one_hot_encoder.transform(y_test.reshape(-1, 1))
#y_test = pd.DataFrame(data=y_test, columns=one_hot_encoder.categories_)

y_val = one_hot_encoder.transform(y_val.reshape(-1, 1))
#y_val = pd.DataFrame(data=y_val, columns=one_hot_encoder.categories_)


print("Shape of y_train:", y_train.shape)

print("Shape of y_test:", y_test.shape)

print("Shape of y_val:", y_val.shape)

In [ ]:
y_train

In [ ]:
y_test_cat = pd.DataFrame(data=y_test, columns=one_hot_encoder.categories_)

In [ ]:
y_test_cat

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Activation
from tensorflow.keras import optimizers
from tensorflow.keras.layers import BatchNormalization, Dropout

In [ ]:
model = Sequential()
model.add(Flatten())
model.add(Dense(10000,kernel_initializer='he_normal'))
model.add(BatchNormalization())                    
model.add(Activation('relu'))    
model.add(Dense(5000,kernel_initializer='he_normal'))
model.add(BatchNormalization())                    
model.add(Activation('relu'))    
model.add(Dense(1000,kernel_initializer='he_normal'))
model.add(BatchNormalization())                    
model.add(Activation('relu'))    
model.add(Dense(500,kernel_initializer='he_normal'))
model.add(BatchNormalization())                    
model.add(Activation('relu')) 
model.add(Dense(250,kernel_initializer='he_normal'))
model.add(BatchNormalization())                    
model.add(Activation('relu'))
model.add(Dense(125,kernel_initializer='he_normal'))
model.add(BatchNormalization())                    
model.add(Activation('relu'))
model.add(Dense(32,kernel_initializer='he_normal'))
model.add(BatchNormalization())                    
model.add(Activation('relu'))
model.add(Dense(12,kernel_initializer='he_normal'))
model.add(Activation('softmax'))

#updating learning rate
adam = optimizers.Adam(lr=0.009, decay=1e-5)
# Compile the model
model.compile(loss="categorical_crossentropy", metrics=["accuracy"], optimizer="adam")

# Fit the model
model.fit(x=X_train, y=y_train, batch_size=50, epochs=10, validation_data=(X_val, y_val))

In [ ]:
results = model.evaluate(X_test, y_test)
print('Accuracy: %f ' % (results[1]*100))
print('Loss: %f' % results[0])

- Acheived a accuracy more than 32% on Test dataset as compared to 24% on KNN
- Will try to further finetune the model

In [ ]:
Y_pred_test_cls = (model.predict(X_test) > 0.5).astype("int32")

In [ ]:
plt.figure(figsize=(2,2))
plt.imshow(X_test[300].reshape(32,32,3))
plt.show()

print('Label - one hot encoded: ',y_test_cat.iloc[300] )
print('Actual Label - one hot encoded:  ', y_test[300])
print('Predicted Label - one hot encoded: ',Y_pred_test_cls[300] )

#### NN Approach 2

In [ ]:
#Reducing complexity

In [ ]:
model = Sequential()
model.add(Flatten())
model.add(Dense(1000,kernel_initializer='he_normal'))
model.add(BatchNormalization())                    
model.add(Activation('relu')) 
model.add(Dense(500,kernel_initializer='he_normal'))
model.add(BatchNormalization())                    
model.add(Activation('relu'))  
model.add(Dense(250,kernel_initializer='he_normal'))
model.add(BatchNormalization())                    
model.add(Activation('relu'))
model.add(Dense(125,kernel_initializer='he_normal'))
model.add(BatchNormalization())                    
model.add(Activation('relu'))
model.add(Dense(32,kernel_initializer='he_normal'))
model.add(BatchNormalization())                    
model.add(Activation('relu'))
model.add(Dense(12,kernel_initializer='he_normal'))
model.add(Activation('softmax'))

#updating learning rate
adam = optimizers.Adam(lr=0.009, decay=1e-6)
# Compile the model
model.compile(loss="categorical_crossentropy", metrics=["accuracy"], optimizer="adam")

# Fit the model
history=model.fit(x=X_train, y=y_train, batch_size=50, epochs=25, validation_data=(X_val, y_val))

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend(loc=0)
plt.figure()


plt.show()

- We can notice the train accuracies are increasing close to 100% but validation accuracies are around 40% only
- We will try to augment the train data and train it again so that we will get less overfitted model

In [ ]:
results = model.evaluate(X_test, y_test)
print('Accuracy: %f ' % (results[1]*100))
print('Loss: %f' % results[0])



Y_pred_test_cls = (model.predict(X_test) > 0.5).astype("int32")

plt.figure(figsize=(2,2))
plt.imshow(X_test[300].reshape(32,32,3))
plt.show()

print('Label - one hot encoded: \n',y_test_cat.iloc[300] )
print('Actual Label - one hot encoded:  ', y_test[300])
print('Predicted Label - one hot encoded: ',Y_pred_test_cls[300] )

- Test data set accuracies are around 35%

In [ ]:
import tensorflow as tf

In [ ]:
#Reimporting data
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2)

X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, random_state=42, test_size=0.5)

In [ ]:
#from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import OneHotEncoder


one_hot_encoder = OneHotEncoder(sparse=False)
one_hot_encoder.fit(y_train.reshape(-1, 1))

y_train = one_hot_encoder.transform(y_train.reshape(-1, 1))
#y_train = pd.DataFrame(data=y_train, columns=one_hot_encoder.categories_)

y_test = one_hot_encoder.transform(y_test.reshape(-1, 1))
#y_test = pd.DataFrame(data=y_test, columns=one_hot_encoder.categories_)

y_val = one_hot_encoder.transform(y_val.reshape(-1, 1))
#y_val = pd.DataFrame(data=y_val, columns=one_hot_encoder.categories_)


print("Shape of y_train:", y_train.shape)

print("Shape of y_test:", y_test.shape)

print("Shape of y_val:", y_val.shape)

In [ ]:
y_train

In [ ]:
y_test_cat = pd.DataFrame(data=y_test, columns=one_hot_encoder.categories_)

- ImageDataGenerator is a preprocessing utility to generate training and testing data with common data augmentation techniques.
- Used to generate training data from Images stored in hierarchical directory structures For more options of ImageDataGenerator go to https://keras.io/preprocessing/image/
    - Rotaion Range -20
    - Width Shift Range - 0.2
    - Height Shift Range - 0.2
    - Zoom Range - 0.4 to 1.5
    - Horizontal flip - True
    - Vertical Flip - True

In [ ]:
datagen= tf.keras.preprocessing.image.ImageDataGenerator(rotation_range=20,
                                                         width_shift_range=0.2,
                                                         height_shift_range=0.2,
                                                         zoom_range=[0.4,1.5],
                                                         horizontal_flip=True,
                                                         vertical_flip=True)

datagen.fit(X_train)

In [ ]:

model = Sequential()
model.add(Flatten())
model.add(Dense(10000,kernel_initializer='he_normal'))
model.add(BatchNormalization())                    
model.add(Activation('relu')) 
model.add(Dense(1000,kernel_initializer='he_normal'))
model.add(BatchNormalization())                    
model.add(Activation('relu')) 
model.add(Dense(500,kernel_initializer='he_normal'))
model.add(BatchNormalization())                    
model.add(Activation('relu'))  
model.add(Dense(250,kernel_initializer='he_normal'))
model.add(BatchNormalization())                    
model.add(Activation('relu'))
model.add(Dense(125,kernel_initializer='he_normal'))
model.add(BatchNormalization())                    
model.add(Activation('relu'))
model.add(Dense(32,kernel_initializer='he_normal'))
model.add(BatchNormalization())                    
model.add(Activation('relu'))
model.add(Dense(12,kernel_initializer='he_normal'))
model.add(Activation('softmax'))

#updating learning rate
adam = optimizers.Adam(lr=0.01, decay=1e-6)
# Compile the model
model.compile(loss="categorical_crossentropy", metrics=["accuracy"], optimizer="adam")

#Saving the best model using model checkpoint callback
model_checkpoint=tf.keras.callbacks.ModelCheckpoint('plantspecies_NN_model.h5', #where to save the model
                                                    save_best_only=True, 
                                                    monitor='val_accuracy', 
                                                    mode='max', 
                                                    verbose=1)

history= model.fit_generator(datagen.flow(X_train, y_train, batch_size=16),  
                  epochs=25, 
                  validation_data=(X_val, y_val),
                  callbacks = [model_checkpoint])
history

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend(loc=0)
plt.figure()


plt.show()

- With This model we can see the train and validation accuracies are close by and hence not a overfitted model
- Still the accuracy results are less than 50%.

In [ ]:
results = model.evaluate(X_test, y_test)
print('Accuracy: %f ' % (results[1]*100))
print('Loss: %f' % results[0])

In [ ]:
results = model.evaluate(X_test, y_test)
print('Accuracy: %f ' % (results[1]*100))
print('Loss: %f' % results[0])



Y_pred_test_cls = (model.predict(X_test) > 0.5).astype("int32")

plt.figure(figsize=(2,2))
plt.imshow(X_test[200].reshape(32,32,3))
plt.show()

print('Label - one hot encoded: \n',y_test_cat.iloc[200] )
print('Actual Label - one hot encoded:  ', y_test[200])
print('Predicted Label - one hot encoded: ',Y_pred_test_cls[200] )

- With Neural network we were able to improve test set predictions to close to 40%.
- Even with NN we are struggling for image identification with high accuracies.
- Epochs were limited at 25 only as we will be comparing the same with CNN on similar grounds. We may acheive a bit higher accuracies with more epochs.
- NN is able to provide better result compared to KNN but it has high computation requirement and huge data to work with.

### Image Classification with CNN

In [ ]:
from tensorflow.keras.models import Sequential  # initial NN
from tensorflow.keras.layers import Dense, Dropout # construct each layer
from tensorflow.keras.layers import Conv2D # swipe across the image by 1
from tensorflow.keras.layers import MaxPooling2D # swipe across by pool size
from tensorflow.keras.layers import Flatten, GlobalAveragePooling2D
from tensorflow import keras

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2)

X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, random_state=42, test_size=0.5)

In [ ]:
#from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import OneHotEncoder


one_hot_encoder = OneHotEncoder(sparse=False)
one_hot_encoder.fit(y_train.reshape(-1, 1))

y_train = one_hot_encoder.transform(y_train.reshape(-1, 1))
#y_train = pd.DataFrame(data=y_train, columns=one_hot_encoder.categories_)

y_test = one_hot_encoder.transform(y_test.reshape(-1, 1))
#y_test = pd.DataFrame(data=y_test, columns=one_hot_encoder.categories_)

y_val = one_hot_encoder.transform(y_val.reshape(-1, 1))
#y_val = pd.DataFrame(data=y_val, columns=one_hot_encoder.categories_)


print("Shape of y_train:", y_train.shape)

print("Shape of y_test:", y_test.shape)

print("Shape of y_val:", y_val.shape)

In [ ]:
y_train

In [ ]:
y_test_cat = pd.DataFrame(data=y_test, columns=one_hot_encoder.categories_)

In [ ]:
y_test_cat

In [ ]:


model = Sequential()
model.add(Conv2D(64, (5, 5), activation='relu', input_shape=(32, 32, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())



model.add(Dense(12, activation='softmax'))

model.summary()

# compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


history=model.fit(x=X_train, y=y_train, batch_size=50, epochs=25, validation_data=(X_val, y_val))
history

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend(loc=0)
plt.figure()


plt.show()

- With a very basic CNN model also we can see the validation accuracies are close to 70%

In [ ]:
results = model.evaluate(X_test, y_test)
print('Accuracy: %f ' % (results[1]*100))
print('Loss: %f' % results[0])



Y_pred_test_cls = (model.predict(X_test) > 0.5).astype("int32")

plt.figure(figsize=(2,2))
plt.imshow(X_test[300].reshape(32,32,3))
plt.show()

print('Label - one hot encoded: \n',y_test_cat.iloc[300] )
print('Actual Label - one hot encoded:  ', y_test[300])
print('Predicted Label - one hot encoded: ',Y_pred_test_cls[300] )

#### Adding few additional layers in model for it to capture more features

In [ ]:


model = Sequential()
model.add(Conv2D(32, (5, 5), activation='relu', input_shape=(32, 32, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(BatchNormalization()) 

model.add(Conv2D(filters=64, kernel_size=4, padding='same', activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(BatchNormalization()) 

model.add(Conv2D(filters=128, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.4))

model.add(Conv2D(filters=256, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.4))

model.add(Flatten())
model.add(Dense(12, activation='softmax'))

model.summary()

#updating learning rate
adam = optimizers.Adam(lr=0.009, decay=1e-6)

# compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

#Saving the best model using model checkpoint callback
model_checkpoint=tf.keras.callbacks.ModelCheckpoint('plantspecies_CNN_model.h5', #where to save the model
                                                    save_best_only=True, 
                                                    monitor='val_accuracy', 
                                                    mode='max', 
                                                    verbose=1)

history=model.fit(x=X_train, y=y_train, 
                  batch_size=50, 
                  epochs=25, 
                  validation_data=(X_val, y_val),
                  callbacks = [model_checkpoint])
history

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend(loc=0)
plt.figure()


plt.show()

- We can observe now the validation accuracies are increased closed to 80%
- still there is a noticable gap betweeen train and validation accuracies, will try to close the gap in further models

In [ ]:
results = model.evaluate(X_test, y_test)
print('Accuracy: %f ' % (results[1]*100))
print('Loss: %f' % results[0])

- Test data set accuracies are immproved closed to 80%

- Traing the model with augmented data set and 500 epochs

- ImageDataGenerator is a preprocessing utility to generate training and testing data with common data augmentation techniques.
- Used to generate training data from Images stored in hierarchical directory structures For more options of ImageDataGenerator go to https://keras.io/preprocessing/image/
    - Rotaion Range -20
    - Width Shift Range - 0.2
    - Height Shift Range - 0.2
    - Zoom Range - 0.4 to 1.5
    - Horizontal flip - True
    - Vertical Flip - True

In [ ]:
datagen= tf.keras.preprocessing.image.ImageDataGenerator(rotation_range=20,
                                                         width_shift_range=0.2,
                                                         height_shift_range=0.2,
                                                         zoom_range=[0.4,1.5],
                                                         horizontal_flip=True,
                                                         vertical_flip=True)

datagen.fit(X_train)

In [ ]:

model = Sequential()
model.add(Conv2D(32, (5, 5), activation='relu', input_shape=(32, 32, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(BatchNormalization()) 

model.add(Conv2D(filters=64, kernel_size=4, padding='same', activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(BatchNormalization()) 

model.add(Conv2D(filters=128, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.4))

model.add(Conv2D(filters=256, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.4))

model.add(Flatten())
model.add(Dense(12, activation='softmax'))

model.summary()

#updating learning rate
adam = optimizers.Adam(lr=0.009, decay=1e-6)

# compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

#Saving the best model using model checkpoint callback
model_checkpoint=tf.keras.callbacks.ModelCheckpoint('plantspecies_CNN_model.h5', #where to save the model
                                                    save_best_only=True, 
                                                    monitor='val_accuracy', 
                                                    mode='max', 
                                                    verbose=1)

history= model.fit_generator(datagen.flow(X_train, y_train, batch_size=32),  
                  epochs=500, 
                  validation_data=(X_val, y_val))
                  #callbacks = [model_checkpoint])
history

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend(loc=0)
plt.figure()


plt.show()

- We can notice the train and test accuracies are overlaping
- This model is less overfitted and providing us with close to 90% accuracies

In [ ]:
results = model.evaluate(X_test, y_test)
print('Accuracy: %f ' % (results[1]*100))
print('Loss: %f' % results[0])

- Test data set accuracies are improved close to 90%

**Always save the model and its weights after training**

In [ ]:
model.save('./Plant_Species_Classifier_500.h5')

model.save_weights('./Plant_Species_Classifier_weights_500.h5')

#we can save using keras methods and job lib as well

<h2>Let's test the model now</h2>

**Load the pre-trained saved model**

In [ ]:

from tensorflow.keras.models import load_model
import numpy as np
from tensorflow.keras.preprocessing import image

# Load the pre trained model from the HDF5 file saved previously
pretrained_model = load_model('./Plant_Species_Classifier_500.h5')
pretrained_model.load_weights('./Plant_Species_Classifier_weights_500.h5')

In [ ]:
model.summary()

**Testing the model on a test image from one of the test folders**

In [ ]:
results = pretrained_model.evaluate(X_test, y_test)
print('Accuracy: %f ' % (results[1]*100))
print('Loss: %f' % results[0])



Y_pred_test_cls = (pretrained_model.predict(X_test) > 0.5).astype("int32")

plt.figure(figsize=(2,2))
plt.imshow(X_test[300].reshape(32,32,3))
plt.show()

print('Label - one hot encoded: \n',y_test_cat.iloc[300] )
print('Actual Label - one hot encoded:  ', y_test[300])
print('Predicted Label - one hot encoded: ',Y_pred_test_cls[300] )

- With CNN we are able to achieve close to 90% accuracies with train, validation and test data.
- We can notice the train and validation accuracies are close to each other at higher epochs.
- we will be using the same for prediction of rest of the data.

### Read the images from test folder and generate test data set

In [ ]:
#we can not directly use the image, we have to process the image.

from pathlib import Path
from skimage.io import imread
from keras.preprocessing import image
import cv2 as cv
def load_test_files(container_path):
    image_dir = Path(container_path)

    images = []
    flat_data = []
    target = []
    count = 0
    train_img = []
    fileNames = []
    for file in image_dir.iterdir():
        count += 1
        img = imread(file)
        img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
        img_pred = cv.resize(img, (32, 32), interpolation=cv.INTER_AREA)
        img_pred = image.img_to_array(img_pred)
        img_pred = img_pred / 255
        train_img.append(img_pred)
        head, tail = os.path.split(file)
        fileNames.append(tail)
            
    X = np.array(train_img)
    y = np.array(fileNames)
    return X,y

#Using the Keras pre-processing library the image is converted to an array and then normalised.

In [ ]:
X_sub,FileNames_Test=load_test_files(specPathTest)

In [ ]:
X_sub.shape

In [ ]:
FileNames_Test.shape

In [ ]:
FileNames_Test[0]

In [ ]:
Y_pred_test_cls = (pretrained_model.predict(X_sub) > 0.5).astype("int32")

In [ ]:
Y_pred_test_cls[0]

In [ ]:
y_test_cat = pd.DataFrame(data=Y_pred_test_cls, columns=one_hot_encoder.categories_)

In [ ]:
y_test_cat

In [ ]:
y_subm_cat=one_hot_encoder.inverse_transform(y_test_cat)

In [ ]:
y_subm_cat.shape

In [ ]:
y_subm_cat=y_subm_cat.reshape(y_subm_cat.shape[0])

In [ ]:
res=pd.DataFrame({"file":FileNames_Test,"species":y_subm_cat})

In [ ]:
res.head()

In [ ]:
plt.figure(figsize=(2,2))
plt.imshow(X_sub[0].reshape(32,32,3))
plt.show()
print(res.species[0])

In [ ]:
plt.figure(figsize=(2,2))
plt.imshow(X_sub[1].reshape(32,32,3))
plt.show()
print(res.species[1])

In [ ]:
plt.figure(figsize=(2,2))
plt.imshow(X_sub[2].reshape(32,32,3))
plt.show()
print(res.species[2])

In [ ]:
res.reset_index(drop=True, inplace=True)

In [ ]:
res.to_csv("submission_PlantClassification.csv",index=False)

# Project Part 2 : 

### Explain in depth why CNN out performs neural networks which in turn out perform supervised learning models when it comes to image classification.

 - For this question we have to divide the models into two major groups:
     - Machine learning Algorithms
     - Deep Neural Networks (NN & CNN)

 - Machine Learning Algorithms:
     - these algo's learn their mapping from provided input and output i.e. alog learns a function with diff sets of weight which help in predicting the accurate values.
     - For classification algorithms they learns from the term being used "Decision Boundries"
     - These decision boundries determine wether a new point belongs to which class or groups
     - Decision boundries could vary from linear to non-linear and these algo's are very strong to identify any relationship and map it with proper function and weight.
     - Image classification althoug a classification problem but it has much more details and relationships which ML Decision boundries are not able to map and replicate without very high computation and some times even thats not enough and becomme impossible for usinng these algos.
     
 - Deep Neural Netowrks ( ANN & CNN ):
     - Deep Neural networks brought in a different concept called Feature Engineering
         - Feature Extraction
         - Feature Selection
     - In feature extraction, we extract all the required features for our problem statement
     - In feature selection, we select the important features that improve the performance of our deep learning model.
     - By this design change in model is giving us huge advantage over Machine learning algorithms for identifying important features of the image and relationships with outputs which helps in categorizing/predictiong classes more accurately.
     
 - Chalanges for Neural Network:
     - NN amount of weight become unmanagable becuse it uses one perceptron for each input/pixel
     - too many parameters as its fully connected
     - each node is connected to previous and next layer making it very dense and many connections are redundant
     - Translation Invariant - NN behaves differently to shifted version of same image/zoomed/inverted. To make it learn all those you have to feed all varaions of data, which is higly difficult.
     - NN expects an identified object should appear on that specific place only which is never the real world scenario.
     - spatial information is lost when the image is flattened(matrix to vector)
     - This will make image processing difficult as the model will tend to overfit and capture unnecessary relationship
     
 - CNN Advantages:

    - Convolution:
         - feed forward NN wont see any order in their inputs
         - CNN on other hand is better at dealing with multiple kinds of spatial deformations. It take advantage of local spatial coherence of images. 
         - This means that they are able to reduce dramatically the number of operation needed to process an image by using convolution on patches of adjacent pixels, because adjacent pixels together are meaningful. 
         - We also call that local connectivity.
         - convolution in neural networks is operation of finding patterns. It has kernel that with which it basically scan an image and place where kernel have 100% match is a place where pattern matched. 
    - Pooling layers:
        - downscale the image
        - This is possible because we retain throughout the network, features that are organized spatially like an image, and thus downscaling them makes sense as reducing the size of the image. 